<a href="https://colab.research.google.com/github/ShovalBenjer/phantom-reach/blob/main/Image_detection_Model_Comparasion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install torch torchvision mediapipe opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 46.8 MB/s eta 0:00:00


In [6]:
import time
import json
import cv2
import mediapipe as mp
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [7]:
def save_landmarks_to_json(landmarks, filepath):
    with open(filepath, 'w') as f:
        json.dump(landmarks, f)

def load_human36m_and_mpii_dataset(human36m_path, mpii_path):
    human36m_data = load_human36m_dataset(human36m_path)
    mpii_data = load_mpii_dataset(mpii_path)  # Add MPII parsing here
    return {"Human3.6M": human36m_data, "MPII": mpii_data}

class PoseModelEvaluator:
    def __init__(self, models, datasets):
        self.models = models  # Dictionary of models {"ModelName": ModelInstance}
        self.datasets = datasets  # Dictionary of datasets {"DatasetName": Dataset}

    def evaluate(self):
        all_results = {}
        for dataset_name, dataset in self.datasets.items():
            results = {}
            for model_name, model in self.models.items():
                preds, gts, inference_times = [], [], []

                for image, gt in preprocess_data(dataset):
                    start_time = time.time()
                    pred = model.predict(image)
                    end_time = time.time()
                    preds.append(pred)
                    gts.append(gt)
                    inference_times.append(end_time - start_time)

                # Metrics
                pck = evaluate_pck(preds, gts)
                mpjpe = evaluate_mpjpe(preds, gts)
                avg_time = np.mean(inference_times)
                fps = 1 / avg_time if avg_time > 0 else float('inf')

                results[model_name] = {
                    "PCK": pck,
                    "MPJPE": mpjpe,
                    "Avg Inference Time": avg_time,
                    "FPS": fps,
                }
            all_results[dataset_name] = results
        return all_results

class MediaPipePoseModel:
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()

    def predict(self, image):
        results = self.pose.process(image)
        if results.pose_landmarks:
            return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
        else:
            return []

In [9]:
# Example Usage
if __name__ == "__main__":
    # Instantiate models
    models = {
        "OpenPose": OpenPoseModel(),
        "PoseNet": PoseNetModel(),
        "HRNet": HRNetModel(),
        "MediaPipe Pose": MediaPipePoseModel(),
        "YOLOv8": YOLOv8PoseModel()
    }

    # Path to Human3.6M dataset
    dataset_path = "/path/to/human36m/dataset"

    # Initialize evaluator
    evaluator = PoseModelEvaluator(models, dataset_path)

    # Evaluate and print results
    results = evaluator.evaluate()
    for model_name, metrics in results.items():
        print(f"Results for {model_name}:")
        for metric, value in metrics.items():
            print(f"{metric}: {value}")


NameError: name 'OpenPoseModel' is not defined